# Finetune Pix2Struct model on Synthetic Bootstrap dataset

## Setup Envirnoment

In [2]:
pip install transformers==4.36.2


Defaulting to user installation because normal site-packages is not writeable
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: /opt/software-current/2023.06/x86_64/generic/software/Python/3.11.3-GCCcore-12.3.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install -q wandb

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
# pip install torchvision nltk wandb tqdm Pillow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 45.8 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: /opt/software-current/2023.06/x86_64/generic/software/Python/3.11.3-GCCcore-12.3.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import necessary libraries

In [1]:
# from google.colab import drive
import os
import zipfile
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import re
from transformers import Pix2StructForConditionalGeneration, AutoProcessor
import torch
from torch.nn import functional as F
from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup
from pathlib import Path
from nltk import edit_distance
import numpy as np
import wandb
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from torch.utils.data import random_split
import random

## Define variables and parameters

In [2]:
# G_DRIVE_FOLDER = '/content/drive/MyDrive/Datasets/'

# DATASET_NAME = 'synthBootstrap_mini'
# ZIP_NAME = DATASET_NAME + '.zip'
# DESTINATION_FOLDER= 'data/'
# DATASET_FOLDER = DESTINATION_FOLDER + DATASET_NAME

# HTML_FILES_FOLDER = DESTINATION_FOLDER + "html/"
# home/seyeon/data/synthBootstrap_mini/html

FOLDER_CHECKPOINTS = ''
DATASET_NAME = 'synthBootstrap_mini/'
ZIP_NAME = DATASET_NAME + '.zip'
DESTINATION_FOLDER= 'data/'
DATASET_FOLDER = DESTINATION_FOLDER + DATASET_NAME
HTML_FILES_FOLDER = DATASET_FOLDER + "html/"


EXPERIMENT_NAME = "Pix2Struct_SynthBootstrap"

MAX_SENTENCE_LEN = 4096

CHUNK_LENGTH =  1024
CONTEXT_OVERLAP_LENGTH = 256

MAX_PATCHES = 1024

DEBUG = False
VERBOSE = True

BATCH_SIZE = 4
NUM_WARMUP_STEPS = 1000
MAX_EPOCHS = 200
LR = 1e-4
CHECK_VAL_EVERY_N_EPOCH = 5
GRADIENT_CLIP_VAL = 1.0
ACCUMULATE_GRAD_BATCHES = 8 / BATCH_SIZE

TRAIN_SET_PERCENTAGE = 0.88
VALID_SET_PERCENTAGE = 0.02 # Use 20 for validation
# TEST_SET_PERCENTAGE is 1 - TRAIN_SET_PERCENTAGE - VALID_SET_PERCENTAGE # Use 100 for test

RANDOM_SEED = 123

LOAD_FROM_CHECKPOINT = False
LAST_CHECKPOINT_NAME = ""

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
MAX_N_CHUNKS_PER_SENTENCE = 1 + (MAX_SENTENCE_LEN - CHUNK_LENGTH) // (CHUNK_LENGTH - CONTEXT_OVERLAP_LENGTH)
print("MAX_N_CHUNKS_PER_SENTENCE", MAX_N_CHUNKS_PER_SENTENCE)

MAX_N_CHUNKS_PER_SENTENCE 5



## Load Synthetic Bootstrap Dataset

## Load Model and Processor

In [2]:
pip install tokenizers==0.21.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs7521888e86192a7e00000001'


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: /opt/software-current/2023.06/x86_64/generic/software/Python/3.11.3-GCCcore-12.3.0/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

repo_id = "google/pix2struct-base"
processor = AutoProcessor.from_pretrained(repo_id)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

## Create Dataset class

### Preprocessing functions

In [6]:
def round_floats_in_text(text, precision=0):
    # match float numbers with 2 or more decimal places in the text
    pattern = r"\b\d+\.\d{2,}\b"

    def replace(match):
        float_number = float(match.group())
        return f"{float_number:.{precision}f}"

    text = re.sub(pattern, replace, text)
    return text

In [7]:
def remove_html_comments(text):
    # match html comments
    pattern = r"<!--.*?-->"

    text = re.sub(pattern, '', text, flags=re.DOTALL)
    return text

In [8]:
def preprocess_html_file(html_text):
    text_cleaned = html_text.replace('\n', ' ')
    text_cleaned_without_multiple_spaces = re.sub(r'\s+', ' ', text_cleaned)
    text_without_comments = remove_html_comments(text_cleaned_without_multiple_spaces)
    text_without_long_floats = round_floats_in_text(text_without_comments)
    return text_without_long_floats

### Find max sentence length and new unknown tokens

In [3]:
import os
import numpy as np
from transformers import AutoTokenizer

# Load your tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

html_dir = "data/pix2code/html"
token_lengths = []

# Tokenize and collect lengths
for fname in os.listdir(html_dir):
    if fname.endswith(".html"):
        with open(os.path.join(html_dir, fname), "r", encoding="utf-8") as f:
            html = f.read()
            tokens = tokenizer(html, truncation=False, add_special_tokens=True)
            token_lengths.append(len(tokens["input_ids"]))

# Convert to numpy array
token_lengths = np.array(token_lengths)

# Compute statistics
avg_len = np.mean(token_lengths)
std_dev = np.std(token_lengths)
min_len = np.min(token_lengths)
max_len = np.max(token_lengths)

print(f"📊 Average token length: {avg_len:.2f}")
print(f"📐 Standard deviation: ±{std_dev:.2f}")
print(f"📈 Max: {max_len}, 📉 Min: {min_len}")
print(f"💡 Range covering ~68% of data: {avg_len - std_dev:.0f} to {avg_len + std_dev:.0f}")


Token indices sequence length is longer than the specified maximum sequence length for this model (1239 > 512). Running this sequence through the model will result in indexing errors


📊 Average token length: 1209.12
📐 Standard deviation: ±175.31
📈 Max: 1649, 📉 Min: 612
💡 Range covering ~68% of data: 1034 to 1384


In [11]:
# Find max length
all_paths = os.listdir(HTML_FILES_FOLDER)

max_length = 0

# Read text files and add new tokens to dictionary
tokens_to_add = set()
for html_file_path in all_paths:
    with open(HTML_FILES_FOLDER + html_file_path, "r") as reader:
        splitted_text = processor.tokenizer(preprocess_html_file(reader.read())).tokens()
        tokens_to_add = tokens_to_add.union(set(splitted_text))

        # Check if the current sentence has the largest number of tokens
        if len(splitted_text) > max_length:
            max_length = len(splitted_text)

print(f"Max sentence length = {max_length}")

newly_added_num = processor.tokenizer.add_tokens(list(tokens_to_add))
print(f"Number of new tokens = {newly_added_num}")

# Resize the model's token embeddings if there are new tokens
if newly_added_num > 0:
    model.decoder.resize_token_embeddings(len(processor.tokenizer))







# from torch.nn import functional as F
# import torch

# # Resize token embeddings (but don't rely on automatic init)
# old_embeddings = model.decoder.get_input_embeddings().weight.data
# old_vocab_size, hidden_size = old_embeddings.shape

# # Perform resize manually
# model.decoder.resize_token_embeddings(len(processor.tokenizer))

# # New embedding layer
# new_embeddings = model.decoder.get_input_embeddings().weight.data
# new_vocab_size = new_embeddings.size(0)

# # Number of newly added tokens
# num_new_tokens = new_vocab_size - old_vocab_size

# if num_new_tokens > 0:
#     # Get the mean and std of existing embeddings
#     mean = old_embeddings.mean(dim=0)
#     cov = torch.cov(old_embeddings.T)  # Use T to get shape (hidden, hidden)

#     # Sample new embeddings using multivariate normal
#     new_embs = torch.distributions.MultivariateNormal(mean, covariance_matrix=cov).sample((num_new_tokens,))
    
#     # Replace the new part of the embedding matrix
#     new_embeddings[old_vocab_size:] = new_embs


Max sentence length = 3995
Number of new tokens = 543


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


### Split files into training - validation - test sets

In [12]:
random.seed(RANDOM_SEED)
random.shuffle(sorted(all_paths))

train_len = int(TRAIN_SET_PERCENTAGE * len(all_paths))
valid_len = int(VALID_SET_PERCENTAGE * len(all_paths))

train_paths = all_paths[:train_len]
valid_paths = all_paths[train_len:train_len+valid_len]
test_paths = all_paths[train_len+valid_len:]

print(f"TRAIN_SET size = {len(train_paths)}")
print(f"VALID_SET size = {len(valid_paths)}")
print(f"TEST_SET size = {len(test_paths)}")

TRAIN_SET size = 880
VALID_SET size = 20
TEST_SET size = 100


In [13]:
class SythBootstrapTrainingDataset(Dataset):
    # This is a modification of the dataset used for validation and testing
    # In this one the sentences are already split into chunks, already having
    # the context from the previous chunk, empty chunks are discarded
    def __init__(self, root_dir, transform, text_files_paths):

        self.root_dir = root_dir
        self.transform = transform
        self.text_files_paths = text_files_paths

        self.max_patches = MAX_PATCHES
        self.max_length = MAX_SENTENCE_LEN
        self.ignore_id = -100

        self.data = []
        self.images_encoding = []

        for text_file in tqdm(text_files_paths):
            image_file = text_file.replace('.html', '.png')

            # Directly process the text files, and save them in the ram
            # Do the same also for images, if there is enough space in memory
            text_file_path = os.path.join(root_dir + "html/", text_file)
            image_file_path = os.path.join(root_dir + "images/", image_file)

            # Each data entry has the following structure
            # labels, image_encoding_idx, part

            # image_encoding_idx points to an entry of images_encoding, which contains attention_mask and flattened_patches for the image
            # Since a single image is used for multiple slices of the same text, this approach is used to save memory

            # Load image
            image = Image.open(image_file_path).convert('RGB')

            if DEBUG:
                image.show()

            if self.transform:
                image = self.transform(image)

            encoding = processor(images=image, max_patches=self.max_patches, return_tensors="pt")
            encoding = {k:v.squeeze() for k,v in encoding.items()}

            self.images_encoding.append(encoding)
            image_encoding_idx = len(self.images_encoding) - 1

            # Load text
            with open(text_file_path, 'r') as f:
                text = f.read()
                text_cleaned = preprocess_html_file(text)

            if DEBUG:
              print("text:")
              print(text)
              print("\n\n\ntext_cleaned:")
              print(text_cleaned)

            input_ids = processor.tokenizer(
                text_cleaned,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            ).input_ids

            input_ids_slices = []

            start_index = 0
            end_index = CHUNK_LENGTH
            while end_index <= MAX_SENTENCE_LEN:
                input_ids_slices.append(input_ids[:, start_index:end_index])
                start_index = end_index - CONTEXT_OVERLAP_LENGTH
                end_index = start_index + CHUNK_LENGTH

            for part, input_ids_slice in enumerate(input_ids_slices):
                labels = input_ids_slice.squeeze().clone()

                labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token

                # Skip slices with only padding tokens, ignore context from the previous chunk
                if part != 0 and all(x == self.ignore_id for x in labels[CONTEXT_OVERLAP_LENGTH:]):
                    continue

                # labels, image_encoding_idx, part
                # Save them as int32 to save ram memory
                self.data.append((labels.to(torch.int32), image_encoding_idx, part))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        labels, image_encoding_idx, part = self.data[idx]
        encoding = self.images_encoding[image_encoding_idx]
        encoding["labels"] = labels.to(torch.int64)

        return encoding, part

In [14]:
class SythBootstrapDataset(Dataset):
    def __init__(self, root_dir, transform, text_files_paths):

        self.root_dir = root_dir
        self.transform = transform
        self.text_files_paths = text_files_paths

        self.max_patches = MAX_PATCHES
        self.max_length = MAX_SENTENCE_LEN
        self.ignore_id = -100

        self.encodings = []

        for text_file in tqdm(text_files_paths):
            image_file = text_file.replace('.html', '.png')

            # Directly process the text files, and save them in the ram
            # Do the same also for images, if there is enough space in memory
            text_file_path = os.path.join(root_dir + "html/", text_file)
            image_file_path = os.path.join(root_dir + "images/", image_file)

            # Load image
            image = Image.open(image_file_path).convert('RGB')

            if DEBUG:
                image.show()

            if self.transform:
                image = self.transform(image)

            encoding = processor(images=image, max_patches=self.max_patches, return_tensors="pt")
            encoding = {k:v.squeeze() for k,v in encoding.items()}

            # Load text
            with open(text_file_path, 'r') as f:
                text = f.read()
                text_cleaned = preprocess_html_file(text)

            if DEBUG:
              print("text:")
              print(text)
              print("\n\n\ntext_cleaned:")
              print(text_cleaned)

            input_ids = processor.tokenizer(
                text_cleaned,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            ).input_ids

            labels = input_ids.squeeze().clone()
            labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token

            encoding["labels"] = labels.to(torch.int32)

            # For each sample save directly the encoding of both text and image
            self.encodings.append(encoding)

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return self.encodings[idx]

In [16]:
# Transformations for the image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize for pretrained models
])

# Instantiate the CustomDataset
train_dataset = SythBootstrapTrainingDataset(DATASET_FOLDER, transform, train_paths)
val_dataset = SythBootstrapDataset(DATASET_FOLDER, transform, valid_paths)

# Use DataLoader for batching and shuffling
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=False) # Use 10 as batch for testing

100%|██████████| 20/20 [00:01<00:00, 19.51it/s]


In [17]:
print(f"train_dataloader size = {len(train_dataloader)}")
print(f"val_dataloader size = {len(val_dataloader)}")

train_dataloader size = 692
val_dataloader size = 2


## Training

In [18]:
START_TOKEN_ID = PAD_TOKEN_ID = processor.tokenizer.pad_token_id

### Utility functions

In [19]:
def move_to_device(data):
    if isinstance(data, (list,tuple)):
        return [move_to_device(x) for x in data]
    elif isinstance(data, dict):
        return {k: move_to_device(v) for k, v in data.items()}
    elif isinstance(data, torch.Tensor):
        return data.to(DEVICE)
    else:
        return data

In [20]:
def create_extended_attention_mask_for_decoder_with_context(input_shape, attention_mask, part):
    device = attention_mask.device
    batch_size, seq_length = input_shape
    seq_ids = torch.arange(seq_length, device=device)

    causal_mask = seq_ids[None, None, :].repeat(batch_size, seq_length, 1) <= seq_ids[None, :, None]

    # Expand part to have the same shape as the relevant slice of causal_mask
    part_expanded = part.unsqueeze(-1).unsqueeze(-1).expand(-1, seq_length, CONTEXT_OVERLAP_LENGTH)

    # Create a mask with ones where part is not zero
    context_mask = (part_expanded != 0).float()

    # Apply the context_mask to the corresponding part of causal_mask
    causal_mask[:, :, :CONTEXT_OVERLAP_LENGTH] = causal_mask[:, :, :CONTEXT_OVERLAP_LENGTH] * (1 - context_mask) + context_mask

    # in case past_key_values are used we need to add a prefix ones mask to the causal mask
    causal_mask = causal_mask.to(attention_mask.dtype)

    if causal_mask.shape[1] < attention_mask.shape[1]:
        print("!!should not enter here in my case!!")
        prefix_seq_len = attention_mask.shape[1] - causal_mask.shape[1]
        causal_mask = torch.cat(
            [
                torch.ones((batch_size, seq_length, prefix_seq_len), device=device, dtype=causal_mask.dtype),
                causal_mask,
            ],
            axis=-1,
        )

    extended_attention_mask = causal_mask[:, :, :] * attention_mask[:, None, :]
    return extended_attention_mask


In [21]:
def get_attention_mask(decoder_input_ids, part):
    # decoder_attention_mask = (decoder_input_ids.ne(PAD_TOKEN_ID).float())
    decoder_attention_mask = (decoder_input_ids != processor.tokenizer.pad_token_id).long()


    # always attend on first token
    decoder_attention_mask[:, 0] = 1

    # Expand part to have the same shape as the relevant slice of decoder_attention_mask
    part_expanded = part.unsqueeze(-1).expand(-1, CONTEXT_OVERLAP_LENGTH)

    # Create a mask with ones where part is not zero
    context_mask = (part_expanded != 0).float()

    # Apply the context_mask to the corresponding part of decoder_attention_mask
    decoder_attention_mask[:, 0:CONTEXT_OVERLAP_LENGTH] = decoder_attention_mask[:, 0:CONTEXT_OVERLAP_LENGTH] * (1 - context_mask) + context_mask

    return decoder_attention_mask

In [22]:
def shift_right_modified(input_ids, decoder_starting_token_idx):

    # shift inputs to the right
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[..., 1:] = input_ids[..., :-1].clone()
    shifted_input_ids[..., 0] = decoder_starting_token_idx

    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, PAD_TOKEN_ID)

    return shifted_input_ids

In [23]:
def get_decoder_input_ids(labels_chunk, start_id):
    return shift_right_modified(labels_chunk, start_id)

In [24]:
def get_decoder_input_ids_and_attention_mask(labels, part):
    decoder_input_ids = get_decoder_input_ids(labels, START_TOKEN_ID)
    decoder_attention_mask = get_attention_mask(decoder_input_ids, part)
    extended_decoder_attention_mask = create_extended_attention_mask_for_decoder_with_context(decoder_input_ids.shape, decoder_attention_mask, part)

    return decoder_input_ids, extended_decoder_attention_mask

### Main training function

In [25]:
def train_model(config, processor, model, train_dataloader, val_dataloader):
    # Extract configuration values
    lr = config.get("lr")
    max_epochs = config.get("max_epochs")
    num_warmup_steps = config.get("num_warmup_steps")

    model.to(DEVICE)

    optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=lr, weight_decay=1e-05)

    # Use total steps (i.e., max_epochs * length_of_train_data)
    total_steps = max_epochs * len(train_dataloader)
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps=num_warmup_steps,
                                                num_training_steps=total_steps)

    global_step = 0  # to keep track of total steps
    epoch_start = 0

    if LOAD_FROM_CHECKPOINT:
        print("Loading model from checkpoint:", LAST_CHECKPOINT_NAME)
        checkpoint = torch.load(LAST_CHECKPOINT_NAME)
        model.resize_token_embeddings(50244) ### retrain
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        epoch_start = checkpoint["epoch"] + 1
        global_step = checkpoint["global_step"] + 1
        wandb_run_id = checkpoint["wandb_run_id"]

        # Resume the WandB run
        wandb.init(project="Pix2Struct", name="run-" + EXPERIMENT_NAME, config=config,     resume="must", id="iaegqr6z")
    else:
        wandb.init(project="Pix2Struct", name="run-" + EXPERIMENT_NAME, config=config)

    epoch_last = epoch_start + max_epochs - 1
    for epoch in range(epoch_start, epoch_start + max_epochs):
        global_step, moving_avg_loss = training_loop(epoch, train_dataloader, model, config, optimizer, scheduler, global_step, epoch_last)

        if epoch == 0 + epoch_start or epoch == epoch_last or (epoch + 1) % config.get("check_val_every_n_epoch") == 0:
            avg_bleu_score = testing_loop(val_dataloader, model, processor, config, f"Epoch {epoch}/{epoch_last} - valid loop")

            # Save the model after each validation step
            save_checkpoint(model, optimizer, scheduler, epoch, global_step, wandb.run.id, avg_bleu_score, EXPERIMENT_NAME, FOLDER_CHECKPOINTS)

            if config.get("verbose", False):
                print(f"Moving Avg Loss: {moving_avg_loss:.3f}")
                print(f" Avg Bleu Score: {avg_bleu_score:.2f}")

            wandb.log({"moving_avg_loss": moving_avg_loss, "bleu": avg_bleu_score, **{f'lr_{i}': param_group['lr'] for i, param_group in enumerate(optimizer.param_groups)}})

    wandb.finish()

In [42]:
def training_loop(epoch, train_dataloader, model, config, optimizer, scheduler, global_step, epoch_last):


    model.train()
    train_loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}/{epoch_last} - train loop")

    # Extract configuration values
    accumulate_grad_batches = config.get('accumulate_grad_batches', 1)
    gradient_clip_val = config.get("gradient_clip_val")

    moving_avg_loss = 0
    alpha = 0.1 # Smoothing factor

    for step, batch in train_loop:
        encoding, part = map(move_to_device, batch)
        labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

        decoder_input_ids, decoder_attention_mask = get_decoder_input_ids_and_attention_mask(labels, part)
        
        # outputs = model(labels=labels, flattened_patches=flattened_patches, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask)
        outputs = model(
            labels=labels,
            flattened_patches=flattened_patches,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask
        )

        loss = outputs.loss
        loss.backward()

        if global_step % accumulate_grad_batches == 0 or step == len(train_dataloader) - 1:
            if gradient_clip_val:
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
            optimizer.step()
            optimizer.zero_grad()

        # Update the progress bar
        train_loop.set_postfix({'loss': loss.item()}, refresh=True)

        scheduler.step()
        global_step += 1

        # Update the moving average loss
        moving_avg_loss = loss.item() if moving_avg_loss == 0 else alpha * loss.item() + (1 - alpha) * moving_avg_loss

        # Log Loss after each step
        wandb.log({"loss": loss.item()})

    return global_step, moving_avg_loss

In [43]:
# def testing_loop(testing_dataloader, model, processor, config, description):
#     model.eval()
#     bleu_scores = []

#     with torch.no_grad():
#         test_loop = tqdm(enumerate(testing_dataloader), total=len(testing_dataloader), desc=description)
#         for i, batch in test_loop:
#             encoding = move_to_device(batch)
#             labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

#             # Initialize total_outputs with zeros
#             total_outputs = None
#             context_from_last = None

#             # Initialize a mask to track which sentences are finished
#             finished_sentences_mask = torch.zeros(flattened_patches.size(0), dtype=torch.bool, device=flattened_patches.device)

#             for iteration in range(MAX_N_CHUNKS_PER_SENTENCE):

#                 generate_args = {
#                     "flattened_patches": flattened_patches[~finished_sentences_mask],
#                     "attention_mask": attention_mask[~finished_sentences_mask],
#                     "max_new_tokens": CHUNK_LENGTH - (CONTEXT_OVERLAP_LENGTH if iteration else 0),
#                 }

#                 if iteration and context_from_last is not None:
#                     generate_args["decoder_input_ids"] = context_from_last[~finished_sentences_mask]

#                 outputs = model.generate(**generate_args)

#                 # Remove context overlap only from the second iteration onwards
#                 new_chunks = outputs if iteration == 0 else outputs[:, CONTEXT_OVERLAP_LENGTH:]

#                 if iteration == 0:
#                     total_outputs = new_chunks
#                 else:
#                     # Update total_outputs by concatenating new chunks
#                     new_chunks_with_padding_chunks = torch.full((flattened_patches.shape[0], new_chunks.shape[1]), PAD_TOKEN_ID, dtype=new_chunks.dtype, device=new_chunks.device)
#                     new_chunks_with_padding_chunks[~finished_sentences_mask] = new_chunks
#                     total_outputs = torch.cat((total_outputs, new_chunks_with_padding_chunks), dim=1)

#                 # Update the finished_sentences_mask
#                 finished_sentences_mask[~finished_sentences_mask] |= (outputs == processor.tokenizer.eos_token_id).any(dim=1)

#                 # If all sentences are finished, exit the loop
#                 if finished_sentences_mask.all():
#                     break

#                 if outputs.shape[1] < CHUNK_LENGTH:
#                     print("ERROR: !! should have already exited because all sentences reached the end!!")

#                 # -1 because it will put in front a START_TOKEN automatically
#                 context_from_last = total_outputs[:, -(CONTEXT_OVERLAP_LENGTH-1):]

#             predictions = processor.tokenizer.batch_decode(total_outputs, skip_special_tokens=True)

#             labels[labels == -100] = 0
#             answers = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

#             bleu_scores += [corpus_bleu([[answer]], [pred], smoothing_function=SmoothingFunction().method4) for pred, answer in zip(predictions, answers)]

#             avg_bleu_score = np.mean(bleu_scores)
#             test_loop.set_postfix(bleu_score=avg_bleu_score)

#             if config.get("verbose", False):
#                 for pred, answer, bleu_score in zip(predictions, answers, bleu_scores):
#                     tqdm.write(f"\nPrediction: {pred}\n    Answer: {answer}\n      Bleu: {bleu_score:.2f}")


#     return avg_bleu_score


In [44]:
# def testing_loop(testing_dataloader, model, processor, config, description):
#     model.eval()
#     bleu_scores = []

#     with torch.no_grad():
#         test_loop = tqdm(enumerate(testing_dataloader), total=len(testing_dataloader), desc=description)
#         for i, batch in test_loop:
#             encoding = move_to_device(batch)
#             labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

#             # Initialize total_outputs with zeros
#             total_outputs = None
#             context_from_last = None

#             # Initialize a mask to track which sentences are finished
#             finished_sentences_mask = torch.zeros(flattened_patches.size(0), dtype=torch.bool, device=flattened_patches.device)

#             for iteration in range(MAX_N_CHUNKS_PER_SENTENCE):

#                 generate_args = {
#                     "flattened_patches": flattened_patches[~finished_sentences_mask],
#                     "attention_mask": attention_mask[~finished_sentences_mask],
#                     "max_new_tokens": CHUNK_LENGTH - (CONTEXT_OVERLAP_LENGTH if iteration else 0),
#                 }

#                 if iteration and context_from_last is not None:
#                     generate_args["decoder_input_ids"] = context_from_last[~finished_sentences_mask]

#                 outputs = model.generate(**generate_args)

#                 # Remove context overlap only from the second iteration onwards
#                 new_chunks = outputs if iteration == 0 else outputs[:, CONTEXT_OVERLAP_LENGTH:]

#                 if iteration == 0:
#                     total_outputs = new_chunks
#                 else:
#                     # Update total_outputs by concatenating new chunks
#                     new_chunks_with_padding_chunks = torch.full((flattened_patches.shape[0], new_chunks.shape[1]), PAD_TOKEN_ID, dtype=new_chunks.dtype, device=new_chunks.device)
#                     new_chunks_with_padding_chunks[~finished_sentences_mask] = new_chunks
#                     total_outputs = torch.cat((total_outputs, new_chunks_with_padding_chunks), dim=1)

#                 # Update the finished_sentences_mask
#                 finished_sentences_mask[~finished_sentences_mask] |= (outputs == processor.tokenizer.eos_token_id).any(dim=1)

#                 # If all sentences are finished, exit the loop
#                 if finished_sentences_mask.all():
#                     break

#                 if outputs.shape[1] < CHUNK_LENGTH:
#                     print("ERROR: !! should have already exited because all sentences reached the end!!")

#                 # -1 because it will put in front a START_TOKEN automatically
#                 context_from_last = total_outputs[:, -(CONTEXT_OVERLAP_LENGTH-1):]

#             predictions = processor.tokenizer.batch_decode(total_outputs, skip_special_tokens=True)

#             labels[labels == -100] = 0
#             answers = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

#             bleu_scores += [corpus_bleu([[answer]], [pred], smoothing_function=SmoothingFunction().method4) for pred, answer in zip(predictions, answers)]

#             avg_bleu_score = np.mean(bleu_scores)
#             test_loop.set_postfix(bleu_score=avg_bleu_score)

#             if config.get("verbose", False):
#                 for pred, answer, bleu_score in zip(predictions, answers, bleu_scores):
#                     tqdm.write(f"\nPrediction: {pred}\n    Answer: {answer}\n      Bleu: {bleu_score:.2f}")


#     return avg_bleu_score


In [45]:
def save_checkpoint(model, optimizer, scheduler, epoch, global_step, wandb_run_id, avg_bleu_score, experiment_name, folder_path):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "epoch": epoch,
        "global_step": global_step,
        'wandb_run_id': wandb_run_id
    }
    model_name = f"{experiment_name}_epoch[{epoch}]_bleu[{avg_bleu_score:.2f}].pth"
    torch.save(checkpoint, folder_path + model_name)


In [46]:
config = {
          "batch_size": BATCH_SIZE,
          "num_warmup_steps": NUM_WARMUP_STEPS,
          "max_epochs": MAX_EPOCHS,
          "lr": LR,
          "check_val_every_n_epoch": CHECK_VAL_EVERY_N_EPOCH,
          "gradient_clip_val": GRADIENT_CLIP_VAL,
          "accumulate_grad_batches": ACCUMULATE_GRAD_BATCHES,
          "verbose": VERBOSE,
}

In [47]:
def validate_config(config):
    # Check required keys
    required_keys = [
        "batch_size",
        "num_warmup_steps",
        "max_epochs",
        "lr",
        "check_val_every_n_epoch",
        "gradient_clip_val",
        "accumulate_grad_batches",
        "verbose"
    ]
    for key in required_keys:
        if key not in config:
            raise ValueError(f"Key '{key}' must be present in the configuration.")

    # Check that values are in expected ranges
    if config["batch_size"] <= 0:
        raise ValueError("batch_size must be positive.")
    if config["num_warmup_steps"] < 0:
        raise ValueError("num_warmup_steps must be non-negative.")
    if config["max_epochs"] <= 0:
        raise ValueError("max_epochs must be positive.")
    if config["lr"] <= 0:
        raise ValueError("Learning rate must be positive.")
    if config["check_val_every_n_epoch"] <= 0:
        raise ValueError("check_val_every_n_epoch must be positive.")
    if config["gradient_clip_val"] < 0:
        raise ValueError("gradient_clip_val must be non-negative.")
    if config["accumulate_grad_batches"] <= 0:
        raise ValueError("accumulate_grad_batches must be positive.")
    if not isinstance(config["verbose"], bool):
        raise ValueError("verbose must be a boolean value.")


In [48]:
validate_config(config)
print(config)

{'batch_size': 4, 'num_warmup_steps': 1000, 'max_epochs': 200, 'lr': 0.0001, 'check_val_every_n_epoch': 5, 'gradient_clip_val': 1.0, 'accumulate_grad_batches': 2.0, 'verbose': True}


In [49]:
# checkpoint = torch.load('checkpointsPix2Struct_SynthBootstrap_1000_Complete_epoch[19]_bleu[0.87].pth')
# print("Saved WandB run ID:", checkpoint['wandb_run_id'])


In [50]:
train_model(config, processor, model, train_dataloader, val_dataloader)

Epoch 0/199 - train loop:   0%|          | 0/692 [00:00<?, ?it/s]


RuntimeError: expand(torch.cuda.FloatTensor{[4, 4, 1, 1024, 1024]}, size=[4, 1, 1024, 1024]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)